In [ ]:
import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"hourly": ["relative_humidity_2m", "cloud_cover", "precipitation", "dew_point_2m", "visibility", "wind_gusts_10m", "temperature_2m"],
	"forecast_hours": 1,
	"wind_speed_unit": "mph",
	"temperature_unit": "fahrenheit",
	"precipitation_unit": "inch",
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_relative_humidity_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(1).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(2).ValuesAsNumpy()
hourly_dew_point_2m = hourly.Variables(3).ValuesAsNumpy()
hourly_visibility = hourly.Variables(4).ValuesAsNumpy()
hourly_wind_gusts_10m = hourly.Variables(5).ValuesAsNumpy()
hourly_temperature_2m = hourly.Variables(6).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end =  pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["precipitation"] = hourly_precipitation
hourly_data["dew_point_2m"] = hourly_dew_point_2m
hourly_data["visibility"] = hourly_visibility
hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m
hourly_data["temperature_2m"] = hourly_temperature_2m

hourly_dataframe = pd.DataFrame(data = hourly_data)
hourly_dataframe["seeing_quality"] = None
print(hourly_dataframe.head())
pd.DataFrame.to_csv(hourly_dataframe, "hourly_data.csv", index = False)

Coordinates: 52.52000045776367°N 13.419998168945312°E
Elevation: 38.0 m asl
Timezone difference to GMT+0: 0s
                       date  relative_humidity_2m  cloud_cover  precipitation  \
0 2025-11-23 22:00:00+00:00                  73.0        100.0            0.0   

   dew_point_2m     visibility  wind_gusts_10m  temperature_2m seeing_quality  
0     17.974562  130905.515625       17.448601         25.4219           None  
